In [7]:
### Import necessary depencencies

import gzip
import json
import multiprocessing
import sqlite3
from multiprocessing import Process

import numpy as np
import pandas as pd

import utils
import re
import unicodedata
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
from django.utils.html import strip_tags
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize.toktok import ToktokTokenizer
from textblob import TextBlob
import numpy as np

from contractions import CONTRACTION_MAP
nlp = spacy.load('en_core_web_sm', parse=True, tag=True, entity=True)




In [8]:
con1 = sqlite3.connect('./data/amazon_cellreviews.db')
#check database
sql = " SELECT reviewText,Clean_Review,sentiment FROM reviews limit 10"
dfdb = pd.read_sql_query(sql, con1)
con1.close()

dfdb.head(10)


,reviewText,Clean_Review,sentiment
0,Looks even better in person. Be careful to not...,look even well person careful drop phone often...,1
1,When you don't want to spend a whole lot of ca...,want spend whole lot cash want great deal shop...,1
2,It is a very cute case. None of the jewels hav...,it cute case none jewel fall bow glue well com...,1
3,Another great product that my daughter she use...,another great product daughter use long time i...,1
4,Beautiful quality and outstanding product! Eve...,beautiful quality outstanding product everyone...,1
5,It is such a good case for a low price. I have...,it good case low price i right never problem b...,1
6,Super durable and I get compliments on it dail...,super durable i get compliment daily i find on...,1
7,I have used this case for a couple weeks & so ...,i use case couple week far great little bling ...,1
8,I chose this case because it is so beautiful. ...,i choose case beautiful everyone like ask coul...,1
9,Of all my cases this is the one that I have on...,case one i phone right i love love love fit gr...,1


In [9]:
# NLTK Stop words
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words.remove('no')
stop_words.remove('not')

In [10]:
def review_to_words(reviews):
    for review in reviews:
        yield(gensim.utils.simple_preprocess(str(review), deacc=False))  # deacc=True removes punctuations

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_tokens = []
    texts_str=[]
    
    for sent in texts:
        doc = nlp(" ".join(sent))
        tokens=[token.lemma_ for token in doc if token.pos_ in allowed_postags]
        texts_tokens.append(tokens)
        texts_str.append(' '.join([word for word in tokens]))
        
    return texts_tokens,texts_str

def expand_contractions(data):
    contraction_mapping = CONTRACTION_MAP
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

    def expand_contractions_sentence(sentence):
        def expand_match(contraction):
            match = contraction.group(0)
            first_char = match[0]
            expanded_contraction = contraction_mapping.get(match) \
                if contraction_mapping.get(match) \
                else contraction_mapping.get(match.lower())
            expanded_contraction = first_char + expanded_contraction[1:]
            return expanded_contraction
        expanded_sentence = contractions_pattern.sub(expand_match, sentence)
        return expanded_sentence

    data_expanded = [expand_contractions_sentence(review) for review in data]   
    return data_expanded

In [11]:
    # Convert to list
    data = dfdb['reviewText']

    # Remove new line characters
    data = [re.sub('\s+', ' ', review) for review in data]

    #contractions
    data_expanded=expand_contractions(data)

    # Remove distracting single quotes
    #data = [re.sub("\'", "", review) for review in data]
    
    #tokenize each review into words
    data_words = list(review_to_words(data_expanded))

    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)  # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)


    # Form Bigrams
    data_words_bigrams = [trigram_mod[bigram_mod[doc]] for doc in data_words_nostops]

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized_tokens,data_lemmatized_str = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    dfdb.insert(3, 'Clean_Review1', data_lemmatized_str)
    dfdb.insert(4, 'Clean_Review2', data_lemmatized_tokens)

    import pickle 
    import math 
    object_pi = math.pi 
    filename= './data/amazon_reviews_processed.pickle' 
    with open(filename, "wb+") as f:
        pickle.dump(dfdb, f)
    
    #print(len(dflocal))
    #sender.send(dflocal)

    #print(processName + ' DONE' )
    
    with open(filename, "rb") as f:
        dfdb_from_file = pickle.load(f)
    
    

In [12]:
    dfdb_from_file.head(10)

,reviewText,Clean_Review,sentiment,Clean_Review1,Clean_Review2
0,Looks even better in person. Be careful to not...,look even well person careful drop phone often...,1,look even well person careful drop phone often...,"[look, even, well, person, careful, drop, phon..."
1,When you don't want to spend a whole lot of ca...,want spend whole lot cash want great deal shop...,1,want spend whole lot cash want great deal shop...,"[want, spend, whole, lot, cash, want, great, d..."
2,It is a very cute case. None of the jewels hav...,it cute case none jewel fall bow glue well com...,1,cute case none jewel fall bow glue well come t...,"[cute, case, none, jewel, fall, bow, glue, wel..."
3,Another great product that my daughter she use...,another great product daughter use long time i...,1,great product daughter long time transaction g...,"[great, product, daughter, long, time, transac..."
4,Beautiful quality and outstanding product! Eve...,beautiful quality outstanding product everyone...,1,beautiful quality outstanding product complime...,"[beautiful, quality, outstanding, product, com..."
5,It is such a good case for a low price. I have...,it good case low price i right never problem b...,1,good case low price right never problem lose s...,"[good, case, low, price, right, never, problem..."
6,Super durable and I get compliments on it dail...,super durable i get compliment daily i find on...,1,super durable get compliment daily find charm ...,"[super, durable, get, compliment, daily, find,..."
7,I have used this case for a couple weeks & so ...,i use case couple week far great little bling ...,1,use case couple week far great little bling st...,"[use, case, couple, week, far, great, little, ..."
8,I chose this case because it is so beautiful. ...,i choose case beautiful everyone like ask coul...,1,choose case beautiful like ask could course ab...,"[choose, case, beautiful, like, ask, could, co..."
9,Of all my cases this is the one that I have on...,case one i phone right i love love love fit gr...,1,case phone right love love love fit great pret...,"[case, phone, right, love, love, love, fit, gr..."
